# NLP練習：以中華民國公司法為語料
## 動機
我個人超討厭法律課~~(拖垮我GPA的罪魁禍首)~~，偏偏本系必修法律6學分，我就想：「我怎麼樣也得讓這個討人厭的東西，反過來對我有點用處吧。」於是，便選擇本學期修商事法中的《公司法》作為我的語料。

奇妙的是，若要我為了準備法律考試而讀這些法條，根本**不可能**~~(所以GPA會被拖垮其實也是因為我都裸考XDD)~~；但是當這些法條作為語料，我卻很願意花時間研讀，也順便準備期中考。

所以跟大家分享一個想題目的方向：其實也可以選一個自己討厭的主題來研究，說不定會有不同的收穫。

## 預定目標
3/8 
> 想子主題

3/15
1.   ~~公司法資料爬取~~
2.   ~~語料基本分析(tokens, types)~~
3.   ~~單條法條長度~~
4.   ~~標點符號之間長度~~(資料型態待修正)

3/22
1.   ~~「但書」多寡~~、~~前後語境~~(可再優化)
2.   「於」、「於…內」、「於…前」、「於…後」：前中後的部分各是什麼

3/29
1.   文言用詞：「之」(誰的?代名詞?)、「其」(誰的?代名詞?)、「為」(替?進行?當?)-->需要標記
2.   「準用」：後面的法條，算類似情形數量

4/5
>爬中華人民共和國公司法做logistic regression(??)


In [2]:
import json
import requests
from bs4 import BeautifulSoup as bs
headers = {'user-agant': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [3]:
url = 'https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=J0080001'
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
html = response.text
dom = bs(html, 'html.parser')

In [4]:
import jieba
import nltk
import re
import statistics as s
import pprint


In [5]:
labels = dom.select('div.col-no > a')
articles = dom.select('div.col-data.text-pre')

In [6]:
data = []
tokens = []
deleted = 0
for i in range(0,len(labels)):
    cpn = dict(NUM=labels[i].text.strip(), Content=articles[i].text.strip().replace(' ', ''))
    text = cpn['Content']
    if text == '（刪除）':
        cpn['length'] = 0
        cpn['sub-sentences'] = 0
        cpn['sub-sent lengths'] = 'None'
        cpn['average sub-sent lengths'] = 'None'
        deleted += 1
    else:
        cpn['length'] = len(text)
        subsent = 0
        punc = [ '，', '：', '；', '。']
        for char in text:
            if char in punc:
                subsent += 1
        cpn['sub-sentences'] = subsent
        subsents = re.split(r'[，：；。]\S', text) #\S:為了不要分到最後一個句點
        subsentlen = []
        for ss in subsents:
            subsentlen.append(len(ss))
            cpn['sub-sent lengths'] = subsentlen
            cpn['average sub-sent lengths'] = s.mean(subsentlen)
        segs = jieba.lcut(text)
        for seg in segs:
            tokens.append(seg)
            types = set(tokens)
    data.append(cpn)
freqdict = nltk.FreqDist(tokens)
print('-'*20)
print('total types:', len(types))
print('total tokens:', len(tokens))
print('deleted articles:', deleted)
print('-'*20)
pp = pprint.PrettyPrinter(width=20, compact=True)
pp.pprint(freqdict.most_common(100))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Yulin\AppData\Local\Temp\jieba.cache
Loading model cost 2.284 seconds.
Prefix dict has been built succesfully.


--------------------
total types: 3254
total tokens: 31616
deleted articles: 98
--------------------
[('，', 2323),
 ('。', 1416),
 ('之', 1365),
 ('公司', 1042),
 ('、', 823),
 ('或', 566),
 ('於', 421),
 ('董事', 355),
 ('股東', 319),
 ('為', 273),
 ('股份', 271),
 ('發行', 263),
 ('人', 240),
 ('有', 217),
 ('其', 206),
 ('時', 198),
 ('者', 193),
 ('以上', 188),
 ('得', 184),
 ('前項', 171),
 ('機關', 165),
 ('主管', 164),
 ('及', 160),
 ('重整', 150),
 ('第一', 148),
 ('項', 145),
 ('；', 140),
 ('對', 137),
 ('後', 137),
 ('規定', 135),
 ('出席', 134),
 ('應', 132),
 ('法院', 124),
 ('已', 123),
 ('條', 120),
 ('股票', 118),
 ('章程', 117),
 ('不得', 115),
 ('總數', 113),
 ('決議', 112),
 ('以下', 106),
 ('同意', 104),
 ('內', 103),
 ('債權', 100),
 ('不', 99),
 ('臺', 99),
 ('但', 98),
 ('由', 97),
 ('股東會', 93),
 ('決權', 90),
 ('一', 87),
 ('代表', 86),
 ('監察人', 85),
 ('登記', 83),
 ('與', 83),
 ('事項', 83),
 ('：', 81),
 ('鍰', 80),
 ('罰', 79),
 ('二', 78),
 ('清算人', 78),
 ('依', 77),
 ('會', 73),
 ('係', 69),
 ('通知', 67),
 ('在', 66),
 ('行使', 66),
 ('更', 65),
 

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [8]:
#句長分析
slength = [data[i]['length'] for i in range(0, len(data)) if data[i]['length'] != 0] #不列入被刪除的法條
sscount = [data[i]['sub-sentences'] for i in range(0, len(data)) if data[i]['sub-sentences'] != 0]
avrgsslen = [data[i]['average sub-sent lengths']  for i in range(0, len(data)) if data[i]['average sub-sent lengths'] != 'None']
        
labels = [[slength,'A:單條完整法條句長'], [sscount, 'B:單條法條所含子句數'],[avrgsslen, 'C:各法條中子句的平均句長']]
for label in labels:
    print('-'*10, label[1], '-'*10)
    mu = '%.4f' % s.mean(label[0])
    var = '%.4f' % s.pvariance(label[0])
    std = '%.4f' % s.pstdev(label[0])
    mini = min(label[0])
    maxi = max(label[0])
    print('minimun = ', mini)
    print('maximum = ', maxi)
    print('mean = ', mu)
    print('variance = ', var)
    print('standard deviation = ', std)
    
print('-'*35)
print('→ correlation of A and B =', '%.4f' % np.corrcoef(slength, sscount)[0][1])
print('→ correlation of B and C =', '%.4f' % np.corrcoef(avrgsslen, sscount)[0][1])

---------- A:單條完整法條句長 ----------
minimun =  13
maximum =  860
mean =  123.6920
variance =  11048.3097
standard deviation =  105.1109
---------- B:單條法條所含子句數 ----------
minimun =  1
maximum =  57
mean =  9.1034
variance =  48.9801
standard deviation =  6.9986
---------- C:各法條中子句的平均句長 ----------
minimun =  5
maximum =  41.666666666666664
mean =  11.7810
variance =  12.9324
standard deviation =  3.5962
-----------------------------------
→ correlation of A and B = 0.9536
→ correlation of B and C = 0.0094


In [ ]:
#histogram of single article length
plt.xlabel('Single article length')
plt.ylabel('Times')
plt.title('Histogram of Single Article Length')
plt.hist(slength, bins=80)

In [ ]:
#子句數量直方圖
plt.xlabel('Numbers of Sub-Sentences')
plt.ylabel('Times')
plt.title('Histogram of Sub-Sentences')
plt.hist(sscount, bins = 60)

In [12]:
#但書多寡
#容忍:不在此限、從其規定、不受限制、但...仍
#限制:不適用之、但(...)應、但(...)不得、以下列為限
## §10 matches for both T and L
T = []
L = []
for i in range(0, len(data)):
    text = data[i]['Content']
    mT = re.search(r'但.*?(不在此限?|從其.+?|不受限制?|免.+?)。', text)  
    mL = re.search(r'但.*?(應.+?|不得.+?|不適用之|為限.*?|限制.*?)。', text)
    if mT:
        T.append((mT.group(),data[i]['NUM']))
    if mL:
        L.append((mL.group(), data[i]['NUM']))
print('-'*10,'Tolerance', '-'*10)
print('-'*10,'共',len(T), '個', '-'*10)
ppp = pprint.PrettyPrinter(width=70, compact=True)
ppp.pprint(T)
print('-'*10, 'Limitation', '-'*10)
print('-'*10,'共',len(L), '個', '-'*10)
ppp.pprint(L)

---------- Tolerance ----------
---------- 共 51 個 ----------
[('但判決確定前，已為補正者，不在此限。', '第 9 條'), ('但已辦妥延展登記者，不在此限。', '第 10 條'),
 ('但自所得產生後逾一年者，不在此限。', '第 23 條'),
 ('但公司章程有較高規定者，從其規定：一、無限公司、兩合公司須有全體無限責任股東過半數同意。', '第 29 條'),
 ('但經依第二十九條第一項規定之方式同意者，不在此限。', '第 32 條'),
 ('但章程中訂定由股東中之一人或數人執行業務者，從其訂定。', '第 45 條'),
 ('但自所得產生後逾一年者，不在此限。', '第 54 條'), ('但向公司清償債務時，不在此限。', '第 59 條'),
 ('但本法或章程另有規定或經股東決議，另選清算人者，不在此限。', '第 79 條'),
 ('但清算人有不法行為時，不在此限。', '第 92 條'), ('但法定盈餘公積已達資本總額時，不在此限。', '第 112 條'),
 ('但第五款約定事項，得免予公告。', '第 133 條'),
 ('但公開發行股票之公司，證券主管機關另有規定者，不在此限。', '第 140 條'),
 ('但公開發行股票之公司，證券管理機關另有規定者，不在此限。', '第 161 條'),
 ('但因繼承者，不在此限。', '第 167-2 條'), ('但本法或其他法律另有規定者，不在此限。', '第 168 條'),
 ('但有正當事由經報請主管機關核准者，不在此限。', '第 170 條'),
 ('但公開發行股票之公司，證券主管機關另有規定者，從其規定。', '第 177 條'),
 ('但聲明撤銷前意思表示者，不在此限。', '第 177-2 條'),
 ('但股東會為前條第一項第二款之決議，同時決議解散時，不在此限。', '第 186 條'),
 ('但經表示異議之董事，有紀錄或書面聲明可證者，免其責任。', '第 193 條'),
 ('但公開發行股票之公司，證券管理機關另有規定者，不在此限。', '第 197-1 條'),
 ('但章程有較高之規定者，從其規定。', '第 204 條'),
 ('但公司章程訂定得由其他董事代理者，不在

In [22]:
#這一段看起來非常笨(沒救)
AT = [] #時
IN_time = [] #時間內
IN_range = [] #範圍內
ER = [] #者
AFT = [] #後
BEF = [] #前
EVE = [] #每次/每年/每會計年度
for i in range(0, len(data)):
    text = data[i]['Content']
    num = data[i]['NUM']
    mAT = re.search(r'於.{1,10}時.+?。', text) ## 208-1
    mIN_time = re.search(r'於.{1,20}(日|月|年|期限|任期)內.+?。', text)
    mIN_range = re.search(r'於.{1,20}(範圍|境|法令)內.+?。', text)
    mER = re.search(r'於.{1,10}者.+?。', text)
    mAFT = re.search(r'於.{1,10}後.+?。', text)
    mBEF = re.search(r'於.{1,10}前.+?。', text)
    mEVE = re.search(r'於每.{1,10}.+?。', text)
    if mAT:
        AT.append((mAT.group(), num))
    if mIN_time:
        IN_time.append((mIN_time.group(), num))
    if mIN_range:
        IN_range.append((mIN_range.group(), num))
print('-'*10,'於...時', '-'*10)
print('-'*10,'共',len(AT), '個', '-'*10)
ppp.pprint(AT)
print('-'*10,'於..(時間)..內', '-'*10)
print('-'*10,'共',len(IN_time), '個', '-'*10)
ppp.pprint(IN_time)
print('-'*10,'於..(範圍)..內', '-'*10)
print('-'*10,'共',len(IN_range), '個', '-'*10)
ppp.pprint(IN_range)

---------- 於...時 ----------
---------- 共 36 個 ----------
[('於申請設立登記時或設立登記後三十日內，檢送經會計師查核簽證之文件。', '第 7 條'),
 ('於變更所營事業時，依代碼表規定辦理。', '第 18 條'),
 ('於自己之重大事由時，不問公司定有存續期限與否，均得隨時退股。', '第 65 條'),
 ('於他人時，應得全體股東之同意。', '第 84 條'),
 ('於六個月內完結清算時，清算人得申敘理由，向法院聲請展期。', '第 87 條'),
 ('於破產管理人時，職務即為終了。', '第 89 條'),
 ('於他人時，應通知公司及其他股東，於二十日內，依第一項或第二項之方式，指定受讓人；逾期未指定或指定之受讓人不依同一條件受讓時，視為同意轉讓，並同意修改章程有關股東及其出資額事項。',
  '第 111 條'),
 ('於每會計年度終了時，查閱公司帳目、業務及財產情形；必要時，法院得因有限責任股東之聲請，許其隨時檢查公司帳目、業務及財產之情形。',
  '第 118 條'),
 ('於自己之重大事由時，得經無限責任股東過半數之同意退股，或聲請法院准其退股。', '第 124 條'),
 ('於前項報告有虛偽情事時，各科新臺幣六萬元以下罰金。', '第 145 條'),
 ('於依前項規定提請股東臨時會決議時，準用之。', '第 168-1 條'), ('於必要時召集之。', '第 170 條'),
 ('於公司利益之虞時，不得加入表決，並不得代理他股東行使其表決權。', '第 178 條'),
 ('於價款支付時生效。', '第 187 條'),
 ('於就任前轉讓超過選任當時所持有之公司股份數額二分之一時，或於股東會召開前之停止股票過戶期間內，轉讓持股超過二分之一時，其當選失其效力。',
  '第 197 條'),
 ('於三十日內召開股東臨時會補選之。', '第 201 條'),
 ('於董事會休會時，依法令、章程、股東會決議及董事會決議，以集會方式經常執行董事會職權，由董事長隨時召集，以半數以上常務董事之出席，及出席過半數之決議行之。',
  '第 208 條'),
 ('於公司之行為。前項臨時管理人，法院應囑託主管機關為之登記。', '第 208-1 條'),
 ('於董事

In [ ]:
之 = []
for i in range(0, len(data)):
    text = data[i]['Content']
    num = data[i]['NUM']
    

In [ ]:
# ??章節和條文無法配合
chapters = dom.select('div.h3.char-2')
chapter_list = []
for chapter in chapters:
    chapter_list.append(chapter.text.strip())

In [ ]:
'''
快速建立由兩個清單而成的dictionary
ALL = dict(zip(listA,listB)
'''

In [ ]:
#子句數量與子句長度關係圖

ssdict = {}
for i in range(0, len(data)):
    ssdict_key = data[i]['sub-sentences']
    ssdict_value = data[i]['sub-sent lengths']
    if ssdict_value != 'None':
        if ssdict_key in ssdict.keys():
            ssdict[ssdict_key].extend(ssdict_value)
        else:
            ssdict[ssdict_key] = ssdict_value

for  key in             